# 学習結果の可視化

`pso_lstm_5m.ipynb` または `lstm_5m_fixed.ipynb` で学習後、以下を表示します。
- **学習曲線**: `final_log_pso_lstm_5m.csv` の Epoch と Loss（train / val）
- **訓練データ**: `train_result.npz` の実測 vs 予測（終値・対数収益）
- **テストデータ**: `test_result.npz` の実測 vs 予測（終値・対数収益）

**前提**: メインノートブックで `final_log_pso_lstm_5m.csv` が生成されていること。
実測 vs 予測は、メインノートで `train_result.npz` および `test_result.npz` を保存している場合に表示可能。
保存される実測・予測は **元スケール（逆変換済み）の対数収益** であり、そのまま可視化・指標計算に使用する。

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from pso_lstm_common import compute_metrics

try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams["font.family"] = ["DejaVu Sans", "Yu Gothic", "Meiryo"]

## 1. 学習曲線（Epoch vs Loss）

In [ ]:
log_path = "final_log_pso_lstm_5m.csv"
if os.path.isfile(log_path):
    log_df = pd.read_csv(log_path)
    fig, ax = plt.subplots(figsize=(8, 4))
    if "loss" in log_df.columns:
        ax.plot(log_df["epoch"], log_df["loss"], label="Train Loss")
    if "val_loss" in log_df.columns:
        ax.plot(log_df["epoch"], log_df["val_loss"], label="Val Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss (MSE)")
    ax.legend()
    ax.set_title("学習曲線")
    plt.tight_layout()
    plt.show()
else:
    print(f"{log_path} が見つかりません。先に pso_lstm_5m.ipynb を実行してください。")

## 2. 訓練データ: 実測 vs 予測（終値・対数収益）

In [ ]:
train_npz_path = "train_result.npz"
if os.path.isfile(train_npz_path):
    data = np.load(train_npz_path)
    y_train = data["y_train"]
    y_train_pred = data["y_pred"]
    lc_train = data["lc_train"]

    # 評価指標（対数収益スケール）
    rmse, mae, mape, r2 = compute_metrics(y_train, y_train_pred)
    corr = np.corrcoef(y_train.ravel(), y_train_pred.ravel())[0, 1] if y_train.size > 1 else 0.0
    dir_acc = (np.sign(y_train.ravel()) == np.sign(y_train_pred.ravel())).mean() * 100.0

    print("=== 訓練データ評価指標（対数収益スケール） ===")
    print("【主な指標】")
    print(f"  相関係数:     {corr:.6f}")
    print(f"  方向一致率:   {dir_acc:.2f}%")
    print("【補助指標】")
    print(f"  RMSE:  {rmse:.6f}  MAE: {mae:.6f}")
    print(f"  MAPE:  {mape:.2f}%  R2: {r2:.6f}")

    actual_close = lc_train * np.exp(y_train)
    pred_close = lc_train * np.exp(y_train_pred)
    n = len(actual_close)
    x_axis = np.arange(n)

    # 終値（価格スケール）のグラフ
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x_axis, actual_close, label="実測終値", alpha=0.8)
    ax.plot(x_axis, pred_close, label="予測終値", alpha=0.8)
    ax.set_xlabel("訓練サンプル")
    ax.set_ylabel("終値")
    ax.legend()
    ax.set_title("訓練データ: 実測 vs 予測（終値）")
    plt.tight_layout()
    plt.show()

    # 対数収益のグラフ
    y_train_flat = y_train.ravel()
    y_train_pred_flat = y_train_pred.ravel()
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x_axis, y_train_flat, label="実測対数収益", alpha=0.8)
    ax.plot(x_axis, y_train_pred_flat, label="予測対数収益", alpha=0.8)
    ax.set_yscale("symlog", linthresh=0.0001)
    ax.yaxis.set_major_locator(ticker.SymmetricalLogLocator(base=10, linthresh=0.0001))
    # 補助目盛り（1, 2, 3...のような細かい線）を表示
    ax.yaxis.set_minor_locator(ticker.SymmetricalLogLocator(base=10, linthresh=0.0001, subs=np.arange(2, 10)))
    ax.grid(True, which="both", ls="--", alpha=0.6)
    ax.axhline(0, color="black", lw=1)  # 0の線を目立たせる
    ax.set_xlabel("訓練サンプル")
    ax.set_ylabel("対数収益（対数スケール）")
    ax.legend()
    ax.set_title("訓練データ: 実測 vs 予測（対数収益・合成）")
    plt.tight_layout()
    plt.show()

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
    ax1.plot(x_axis, y_train_flat, color="C0", alpha=0.8)
    ax1.set_ylabel("対数収益（実測）")
    ax1.set_title("実測対数収益")
    ax1.grid(True, alpha=0.3)
    ax2.plot(x_axis, y_train_pred_flat, color="C1", alpha=0.8)
    ax2.set_xlabel("訓練サンプル")
    ax2.set_ylabel("対数収益（予測）")
    ax2.set_title("予測対数収益")
    ax2.grid(True, alpha=0.3)
    plt.suptitle("訓練データ: 実測 vs 予測（対数収益）", y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print(f"{train_npz_path} が見つかりません。訓練結果を保存するには、lstm_5m_fixed.ipynb で train_result.npz を保存してください。")

## 3. テストデータ: 実測 vs 予測（終値・対数収益）

In [ ]:
npz_path = "test_result.npz"
if os.path.isfile(npz_path):
    data = np.load(npz_path)
    y_test = data["y_test"]
    y_pred = data["y_pred"]
    lc_test = data["lc_test"]

    # 評価指標（対数収益スケール）
    rmse, mae, mape, r2 = compute_metrics(y_test, y_pred)
    corr = np.corrcoef(y_test.ravel(), y_pred.ravel())[0, 1] if y_test.size > 1 else 0.0
    dir_acc = (np.sign(y_test.ravel()) == np.sign(y_pred.ravel())).mean() * 100.0

    print("=== テストデータ評価指標（対数収益スケール） ===")
    print("【主な指標】")
    print(f"  相関係数:     {corr:.6f}")
    print(f"  方向一致率:   {dir_acc:.2f}%")
    print("【補助指標】")
    print(f"  RMSE:  {rmse:.6f}  MAE: {mae:.6f}")
    print(f"  MAPE:  {mape:.2f}%  R2: {r2:.6f}")

    actual_close = lc_test * np.exp(y_test)
    pred_close = lc_test * np.exp(y_pred)
    n = len(actual_close)
    x_axis = np.arange(n)

    # 終値（価格スケール）のグラフ
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x_axis, actual_close, label="実測終値", alpha=0.8)
    ax.plot(x_axis, pred_close, label="予測終値", alpha=0.8)
    ax.set_xlabel("テストサンプル")
    ax.set_ylabel("終値")
    ax.legend()
    ax.set_title("テストデータ: 実測 vs 予測（終値）")
    plt.tight_layout()
    plt.show()

    # 対数収益のグラフ
    y_test_flat = y_test.ravel()
    y_pred_flat = y_pred.ravel()
    # 合成グラフ（実測・予測を1枚に）
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x_axis, y_test_flat, label="実測対数収益", alpha=0.8)
    ax.plot(x_axis, y_pred_flat, label="予測対数収益", alpha=0.8)
    ax.set_yscale("symlog", linthresh=0.0001)
    ax.yaxis.set_major_locator(ticker.SymmetricalLogLocator(base=10, linthresh=0.0001))
    # 補助目盛り（1, 2, 3...のような細かい線）を表示
    ax.yaxis.set_minor_locator(
        ticker.SymmetricalLogLocator(base=10, linthresh=0.0001, subs=np.arange(2, 10))
    )
    ax.grid(True, which="both", ls="--", alpha=0.6)
    ax.axhline(0, color="black", lw=1)  # 0の線を目立たせる
    ax.set_xlabel("テストサンプル")
    ax.set_ylabel("対数収益（対数スケール）")
    ax.legend()
    ax.set_title("テストデータ: 実測 vs 予測（対数収益・合成）")
    plt.tight_layout()
    plt.show()
    # 別軸グラフ（つぶれを防ぐ）
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
    ax1.plot(x_axis, y_test_flat, color="C0", alpha=0.8)
    ax1.set_ylabel("対数収益（実測）")
    ax1.set_title("実測対数収益")
    ax1.grid(True, alpha=0.3)
    ax2.plot(x_axis, y_pred_flat, color="C1", alpha=0.8)
    ax2.set_xlabel("テストサンプル")
    ax2.set_ylabel("対数収益（予測）")
    ax2.set_title("予測対数収益")
    ax2.grid(True, alpha=0.3)
    plt.suptitle("テストデータ: 実測 vs 予測（対数収益）", y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print(f"{npz_path} が見つかりません。実測 vs 予測を保存するには、lstm_5m_fixed.ipynb または pso_lstm_5m.ipynb で test_result.npz を保存してください。")